In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from utils import *
from usad import *

### Import data

In [108]:
file = "SCADA_Station1"
num_rows = 30000
dataset = pd.read_csv(f'input/{file}.csv', nrows=num_rows)

dataset["Time"] = pd.to_datetime(dataset["Time"])
dataset["Time"] = dataset["Time"].diff()
dataset["Time"] = pd.to_numeric(dataset["Time"])/1e6 #time in milliseconds
dataset = dataset.drop(index=0)

dataset["label"] = 0

dataset.head(5)

,Time,Address,IOA,Value,label
1,7071.274,1143,11009,38.5,0
2,99.052,1143,11008,60.6,0
3,8113.107,1143,11008,55.0,0
4,753.655,1143,11007,29.1,0
5,3056.284,1143,11008,60.2,0


### Add attacks

In [109]:
attacks = True

if attacks :
    row =5*num_rows/8
    for i in range(1,21):
        dataset.loc[row+i, "Value"] *= 5
        dataset.loc[row+i, "label"] = 1

dataset.iloc[int(row)-2:int(row)+22]

,Time,Address,IOA,Value,label
18749,4557.685,1143,11008,108.6,0
18750,3861.910,1143,11007,63.7,0
18751,9212.814,1143,11008,515.5,1
18752,0.000,1143,11009,421.0,1
18753,954.440,1143,11043,-138.0,1
18754,12872.902,1143,11007,344.0,1
18755,4360.031,1143,11039,64.5,1
18756,1700.169,1143,11009,448.0,1
18757,22641.293,1143,11009,421.5,1
18758,1251.764,1143,11039,23.5,1


### Convert to float

In [110]:
col = lambda column : [ord(char) for char in column]
def convert_to_float(column):
    try:
        return float(column)
    except ValueError:
        return ''.join(map(str, col(str(column))))

for i in list(dataset):
    dataset[i] = dataset[i].apply(convert_to_float)
dataset = dataset.astype(float)

In [111]:
dataset.head(5)

,Time,Address,IOA,Value,label
1,7071.274,1143.0,11009.0,38.5,0.0
2,99.052,1143.0,11008.0,60.6,0.0
3,8113.107,1143.0,11008.0,55.0,0.0
4,753.655,1143.0,11007.0,29.1,0.0
5,3056.284,1143.0,11008.0,60.2,0.0


In [112]:
dataset = dataset.drop(["Time"], axis=1)
dataset.head(5)

,Address,IOA,Value,label
1,1143.0,11009.0,38.5,0.0
2,1143.0,11008.0,60.6,0.0
3,1143.0,11008.0,55.0,0.0
4,1143.0,11007.0,29.1,0.0
5,1143.0,11008.0,60.2,0.0


In [113]:
temp = dataset.drop(["label"], axis=1)
temp.to_csv(f'data/{file}.csv', header=True, index=False)
dataset.to_csv(f'data/{file}_label.csv', header=True, index=False)